In [1]:
import torch
from torchvision.datasets import Imagenette
from torchvision.transforms import transforms
import torch.nn as nn
import matplotlib.pyplot as pyplot
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

## Device Agnostic Code


In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
mean=[0.485, 0.456, 0.406]
std= [0.229, 0.224, 0.225]

In [4]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(227),
    transforms.Normalize(mean,std)
    ]
)

train_data=Imagenette(root='dataset/train',split='train',download=False,transform=transform)
val_data=Imagenette(root='dataset/test',split='val',download=False,transform=transform)


## Making Dataloader

In [5]:
train_dataloader=DataLoader(train_data,batch_size=128,shuffle=True)
val_dataloader=DataLoader(val_data,batch_size=128)

# Building Alexnet Architecture

In [6]:
class Alexnet(nn.Module):
    
    def __init__(self,in_channel=3,out_classes=10):
        super().__init__()

        self.in_channel=in_channel
        self.out_classes=out_classes

        self.model=nn.Sequential(

            # conv->maxpool->conv->maxpool->3*conv->maxpool->3*fc->outputlayer
            nn.Conv2d(self.in_channel,96,11,4),
            nn.ReLU(),
            nn.LocalResponseNorm(5,0.001,0.75,2),
            nn.MaxPool2d(3,2),
            nn.Conv2d(96,256,5,1,padding='same'),
            nn.ReLU(),
            nn.LocalResponseNorm(5,0.001,0.75,2),
            nn.MaxPool2d(3,2),

            nn.Conv2d(256,384,3,1,padding='same'),
            nn.ReLU(),
            nn.Conv2d(384,384,3,1,padding='same'),
            nn.ReLU(),
            nn.Conv2d(384,256,3,1,padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(3,2),

            # fully connected layer
            nn.Flatten(start_dim=1),
            nn.Linear(256*6*6,4096),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.Dropout(0.5),
            nn.Linear(4096,self.out_classes),        

        )

    def forward(self,x):
        return self.model(x)

In [7]:
alexnet=Alexnet()
alexnet.to(device)

Alexnet(
  (model): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=9216, out_features=4096, bias=True)
    (17): Dropout(p=0.5, inplace=False)
    (1

## Train the Model

In [ ]:
optim=torch.optim.SGD(alexnet.parameters(),lr=0.01,momentum=0.9)

In [ ]:
loss_train=[]
loss_test=[]

In [ ]:
for epoch in range(10):
    running_loss=0
    count=0
    train_accuracy=0
    for idx,data in enumerate(train_dataloader):
        count+=1
        optim.zero_grad()

        img,label=data
        logits=alexnet(img.to(device))

        loss=(torch.nn.functional.cross_entropy(logits,label.to(device)))

        loss.backward()
        optim.step()

        running_loss+=loss
        train_accuracy+=(torch.sum(torch.where(logits.argmax(1)==label.to(device), 1, 0)))/logits.shape[0]

        # with torch.no_grad():
        #   for idx,data in enumerate(test_dataloader):
        #     img,label=data
        #     logits=alexnet(img.to(device))
        #     loss_test=(torch.nn.functional.cross_entropy(logits,label.to(device)))




    loss_train.append((running_loss/count).item())


    print(f'Epoch: {epoch} | Training Loss: {(running_loss/count).item(): .3f} | Train-Accuracy: {train_accuracy: .3f}')


In [ ]:
plt.plot(loss_train)

## Loading the Saved Model

In [8]:
alexnet.load_state_dict(torch.load('alexnet.pt'))

<All keys matched successfully>

## Train Accuracy

In [ ]:
# for idx,data in enumerate(train_dataloader):
#     count+=1
#     optim.zero_grad()

#     img,label=data
#     logits=alexnet(img.to(device))

#     loss=(torch.nn.functional.cross_entropy(logits,label.to(device)))

#     loss.backward()
#     optim.step()

#     running_loss+=loss
#     train_accuracy+=(torch.sum(torch.where(logits.argmax(1)==label.to(device), 1, 0)))/logits.shape[0]

In [12]:
for params in alexnet.parameters():
    params.requires_grad=False


alexnet.model[20]=nn.Linear(in_features=4096,out_features=2)


OrderedDict()